In [2]:
from brainlit.utils import upload, session
from pathlib import Path
import napari
import os

## Uploading Brain Images from data in the Octree format.
This notebook demonstrates uploading the 2 lowest-resolution brain volumes and a `.swc` segment file.
The upload destination could easily be set to a url of a cloud data server such as s3.

## 1) Define variables.
 - `source` and `source_segments` are the root directories of the octree-formatted data and swc files.
 - `p` is the prefix string. `file://` indicates a filepath, while `s3://` or `gc://` indicate URLs.
 - `dest` and `dest_segments` are the destinations for the uploads (in this case, filepaths).
 - `num_res` denotes the number of resolutions to upload. 
 
The below paths lead to sample data in the NDD Repo. Alter the below path definitions to point to your own local file locations. 

In [3]:
source = (Path().resolve().parents[2] / "data" / "data_octree").as_posix()
dest = (Path().resolve().parents[2] / "data" / "upload").as_uri()
dest_segments = (Path().resolve().parents[2] / "data" / "upload_segments").as_uri()
dest_annotation = (Path().resolve().parents[2] / "data" / "upload_annotation").as_uri()
num_res = 2

## 2) Upload the image data (.tif)

If the upload fails with the error: `timed out on a chunk on layer index 0. moving on to the next step of pipeline`, re-run the `upload_volumes` function but with the `continue_upload` parameter, which takes `layer index` (the layer index said in the error message) and `image index` (the last succesful image that uploaded).  

For example, if the output failed after image 19, then run 
`upload.upload_volumes(source, dest, num_res, continue_upload = (0, 19))`. Repeat this till all of the upload is complete.


In [ ]:
upload.upload_volumes(source, dest, num_res)

## 3) Upload the segmentation data (.swc)

If uploading a `.swc` file associated with a brain volume, then use upload.py. Otherwise if uploading `swc` files with different name formats, use upload_benchmarking.py

In [ ]:
upload.upload_segments(source, dest_segments, num_res)

## Download the data with NeuroglancerSession and generate labels.

In [ ]:
%%capture
sess = session.NeuroglancerSession(url=dest, url_segments=dest_segments, mip=0)  # create session object object
img, bounds, vertices = sess.pull_vertex_list(2, range(250, 350), expand=True)  # get image containing some data
labels = sess.create_tubes(2, bounds, radius=1)  # generate labels via tube segmentation

## 4) Visualize the data with napari

In [ ]:
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_labels(labels)